In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

In [3]:
df = pd.read_csv('email.csv')  # Replace with your dataset file
df['Label'] = df['Category'].map({'ham':0, "spam":1})
print(df.head)



train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Message'], df['Label'], test_size = 0.2, random_state = 42
)



<bound method NDFrame.head of             Category                                            Message  Label
0                ham  Go until jurong point, crazy.. Available only ...    0.0
1                ham                      Ok lar... Joking wif u oni...    0.0
2               spam  Free entry in 2 a wkly comp to win FA Cup fina...    1.0
3                ham  U dun say so early hor... U c already then say...    0.0
4                ham  Nah I don't think he goes to usf, he lives aro...    0.0
...              ...                                                ...    ...
5568             ham               Will ü b going to esplanade fr home?    0.0
5569             ham  Pity, * was in mood for that. So...any other s...    0.0
5570             ham  The guy did some bitching but I acted like i'd...    0.0
5571             ham                         Rofl. Its true to its name    0.0
5572  {"mode":"full"                                    isActive:false}    NaN

[5573 rows x 3 column

In [4]:
# Tokenize Messages
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


#Tokenize the text data
train_encodings = tokenizer(list(train_texts), truncation = True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding = True, max_length=128)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
import torch

class MessageDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = [int(label) for label in labels]
        
        assert all(0 <= label < 2 for label in self.labels), "Labels must be 0 or 1"

        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        # Fix labels if they're extracted as floats

        return item
    

train_dataset = MessageDataset(train_encodings, list(train_labels))
val_dataset = MessageDataset(val_encodings, list(val_labels))


ValueError: cannot convert float NaN to integer

In [27]:
from transformers import BertForSequenceClassification,  AdamW
from torch.utils.data import DataLoader

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)
model.config.problem_type = "single_label_classification"

train_loader = DataLoader(train_dataset, batch_size=16)
val_loader = DataLoader(val_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


for epoch in range(3):
    model.train()
    for batch in train_loader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs = {key:val.to(device) for key, val in batch.items()}
            outputs = model(**inputs)
            val_loss +=outputs.loss.item()
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss}")
    


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: value cannot be converted to type int64 without overflow

In [19]:
for batch in train_loader:
    print(batch['labels'].shape)  # Should be torch.Size([batch_size])
    print(batch['labels'])        # Should contain integers like [0, 1, 0, ...]
    break


torch.Size([16])
tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [18]:
for batch in train_loader:
    inputs = {key: val.to(device) for key, val in batch.items()}
    outputs = model(**inputs)
    print(outputs.logits.shape)  # Should be torch.Size([batch_size, num_labels])
    break


torch.Size([16, 2])


NameError: name 'labels' is not defined